## I Filter

In [36]:
import os
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
import astropy.units as u
from photutils.aperture import SkyCircularAperture, CircularAperture, SkyCircularAnnulus, aperture_photometry
from astropy.stats import sigma_clipped_stats

# Load APASS catalog CSV file
apass_data = pd.read_csv("/Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/SA 98/apassSA98.csv")

# Define your base path for FITS files
base_path = "/Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/SA 98/calibrated_i"

# Iterate over each file in the directory
for filename in os.listdir(base_path):
    if filename.endswith('.fts'):
        file_path = os.path.join(base_path, filename)
        results = []  # Initialize the results list inside the loop for separate CSV for each file
        try:
            hdulist = fits.open(file_path)
            data = hdulist[0].data
            wcs = WCS(hdulist[0].header)
            star_positions = SkyCoord(ra=apass_data['RA_degrees']*u.deg, dec=apass_data['Dec_degrees']*u.deg)

            # Retrieve airmass from the FITS header
            airmass = hdulist[0].header.get('AIRMASS', 'unknown')

            # Define apertures and convert to pixel coordinates
            apertures = SkyCircularAperture(star_positions, r=0.004 * u.deg)
            pixel_apertures = apertures.to_pixel(wcs)

            # Define annuli and convert to pixel coordinates for background estimation
            annuli = SkyCircularAnnulus(star_positions, r_in=0.005 * u.deg, r_out=0.006 * u.deg)
            pixel_annuli = annuli.to_pixel(wcs)

            # Perform background subtraction using sigma clipping
            bkg_median = []
            for annulus in pixel_annuli:
                mask = annulus.to_mask(method='center')
                annulus_data = mask.multiply(data)
                if np.any(mask.data > 0):  # Ensure data within mask
                    annulus_data_1d = annulus_data[mask.data > 0]
                    _, median_sigclip, _ = sigma_clipped_stats(annulus_data_1d)
                    bkg_median.append(median_sigclip)
                else:
                    bkg_median.append(0)  # No data to process

            bkg_median = np.array(bkg_median)
            phot_table = aperture_photometry(data, pixel_apertures, error=np.sqrt(data), wcs=wcs)
            phot_table['bkg_median'] = bkg_median
            phot_table['aperture_sum_bkg_sub'] = phot_table['aperture_sum'] - bkg_median * np.array([ap.area for ap in pixel_apertures])

            # Adding celestial coordinates
            phot_table['ra_icrs'] = star_positions.ra.degree
            phot_table['dec_icrs'] = star_positions.dec.degree

            # Match photometry results with catalog data
            for i, row in enumerate(phot_table):
                catalog_match = apass_data.iloc[np.argmin(star_positions.separation(star_positions[i]).degree)]
                results.append({
                    'RA': row['ra_icrs'],
                    'DEC': row['dec_icrs'],
                    'Instrumental_I': -2.5 * np.log10(row['aperture_sum_bkg_sub']),
                    'InstrumentalI_err': 1.0857 * row['aperture_sum_err'] / row['aperture_sum'],
                    'Known_I': catalog_match['Ic'],
                    'KnownI_err': catalog_match['Ic_err']
                })

            results_df = pd.DataFrame(results)
            csv_filename = f"IResult_{airmass}.csv"
            csv_file_path = os.path.join(base_path, csv_filename)
            results_df.to_csv(csv_file_path, index=False)
            print(f"Results saved to {csv_file_path}")

        except Exception as e:
            print(f"Failed to process {filename}: {e}")





the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
/var/folders/xh/69tqn8vn1vd5sw0_tn9ksy5w0000gn/T/ipykernel_30578/2503332311.py:66: RuntimeWarning: invalid value encountered in log10
  'Instrumental_I': -2.5 * np.log10(row['aperture_sum_bkg_sub']),


Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/SA 98/calibrated_i/IResult_1.30662842941.csv


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
/var/folders/xh/69tqn8vn1vd5sw0_tn9ksy5w0000gn/T/ipykernel_30578/2503332311.py:66: RuntimeWarning: invalid value encountered in log10
  'Instrumental_I': -2.5 * np.log10(row['aperture_sum_bkg_sub']),


Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/SA 98/calibrated_i/IResult_1.52456664101.csv


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
/var/folders/xh/69tqn8vn1vd5sw0_tn9ksy5w0000gn/T/ipykernel_30578/2503332311.py:66: RuntimeWarning: invalid value encountered in log10
  'Instrumental_I': -2.5 * np.log10(row['aperture_sum_bkg_sub']),
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/SA 98/calibrated_i/IResult_1.54918103305.csv


/var/folders/xh/69tqn8vn1vd5sw0_tn9ksy5w0000gn/T/ipykernel_30578/2503332311.py:66: RuntimeWarning: invalid value encountered in log10
  'Instrumental_I': -2.5 * np.log10(row['aperture_sum_bkg_sub']),
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
/var/folders/xh/69tqn8vn1vd5sw0_tn9ksy5w0000gn/T/ipykernel_30578/2503332311.py:66: RuntimeWarning: invalid value encountered in log10
  'Instrumental_I': -2.5 * np.log10(row['aperture_sum_bkg_sub']),
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/SA 98/calibrated_i/IResult_1.36646202666.csv
Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/SA 98/calibrated_i/IResult_1.85706167933.csv
Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/SA 98/calibrated_i/IResult_1.38404453414.csv


/var/folders/xh/69tqn8vn1vd5sw0_tn9ksy5w0000gn/T/ipykernel_30578/2503332311.py:66: RuntimeWarning: invalid value encountered in log10
  'Instrumental_I': -2.5 * np.log10(row['aperture_sum_bkg_sub']),


## R Filter

In [1]:
import os
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
import astropy.units as u
from photutils.aperture import SkyCircularAperture, CircularAperture, SkyCircularAnnulus, aperture_photometry
from astropy.stats import sigma_clipped_stats

# Load APASS catalog CSV file
apass_data = pd.read_csv("/Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/SA 98/apassSA98.csv")

# Define your base path for FITS files
base_path = "/Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/SA 98/calibrated_r"

# Iterate over each file in the directory
for filename in os.listdir(base_path):
    if filename.endswith('.fts'):
        file_path = os.path.join(base_path, filename)
        results = []  # Initialize the results list inside the loop for separate CSV for each file
        try:
            hdulist = fits.open(file_path)
            data = hdulist[0].data
            wcs = WCS(hdulist[0].header)
            star_positions = SkyCoord(ra=apass_data['RA_degrees']*u.deg, dec=apass_data['Dec_degrees']*u.deg)

            # Retrieve airmass from the FITS header
            airmass = hdulist[0].header.get('AIRMASS', 'unknown')

            # Define apertures and convert to pixel coordinates
            apertures = SkyCircularAperture(star_positions, r=0.004 * u.deg)
            pixel_apertures = apertures.to_pixel(wcs)

            # Define annuli and convert to pixel coordinates for background estimation
            annuli = SkyCircularAnnulus(star_positions, r_in=0.005 * u.deg, r_out=0.006 * u.deg)
            pixel_annuli = annuli.to_pixel(wcs)

            # Perform background subtraction using sigma clipping
            bkg_median = []
            for annulus in pixel_annuli:
                mask = annulus.to_mask(method='center')
                annulus_data = mask.multiply(data)
                if np.any(mask.data > 0):  # Ensure data within mask
                    annulus_data_1d = annulus_data[mask.data > 0]
                    _, median_sigclip, _ = sigma_clipped_stats(annulus_data_1d)
                    bkg_median.append(median_sigclip)
                else:
                    bkg_median.append(0)  # No data to process

            bkg_median = np.array(bkg_median)
            phot_table = aperture_photometry(data, pixel_apertures, error=np.sqrt(data), wcs=wcs)
            phot_table['bkg_median'] = bkg_median
            phot_table['aperture_sum_bkg_sub'] = phot_table['aperture_sum'] - bkg_median * np.array([ap.area for ap in pixel_apertures])

            # Adding celestial coordinates
            phot_table['ra_icrs'] = star_positions.ra.degree
            phot_table['dec_icrs'] = star_positions.dec.degree

            # Match photometry results with catalog data
            for i, row in enumerate(phot_table):
                catalog_match = apass_data.iloc[np.argmin(star_positions.separation(star_positions[i]).degree)]
                results.append({
                    'RA': row['ra_icrs'],
                    'DEC': row['dec_icrs'],
                    'Instrumental_R': -2.5 * np.log10(row['aperture_sum_bkg_sub']),
                    'InstrumentalR_err': 1.0857 * row['aperture_sum_err'] / row['aperture_sum'],
                    'Known_R': catalog_match['Rc'],
                    'KnownR_err': catalog_match['Rc_err']
                })

            results_df = pd.DataFrame(results)
            csv_filename = f"RResult_{airmass}.csv"
            csv_file_path = os.path.join(base_path, csv_filename)
            results_df.to_csv(csv_file_path, index=False)
            print(f"Results saved to {csv_file_path}")

        except Exception as e:
            print(f"Failed to process {filename}: {e}")


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/SA 98/calibrated_r/RResult_1.52639624326.csv
Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/SA 98/calibrated_r/RResult_1.30251975605.csv


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
/var/folders/xh/69tqn8vn1vd5sw0_tn9ksy5w0000gn/T/ipykernel_46446/458396092.py:66: RuntimeWarning: invalid value encountered in log10
  'Instrumental_R': -2.5 * np.log10(row['aperture_sum_bkg_sub']),
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
/var/folders/xh/69tqn8vn1vd5sw0_tn9ksy5w0000gn/T/ipykernel_46446/458396092.py:66: RuntimeWarning: invalid value encountered in log10
  'Instrumental_R': -2.5 * np.log10(row['aperture_sum_bkg_sub']),
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/SA 98/calibrated_r/RResult_1.35593709572.csv
Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/SA 98/calibrated_r/RResult_1.81436066349.csv


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
/var/folders/xh/69tqn8vn1vd5sw0_tn9ksy5w0000gn/T/ipykernel_46446/458396092.py:66: RuntimeWarning: invalid value encountered in log10
  'Instrumental_R': -2.5 * np.log10(row['aperture_sum_bkg_sub']),
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/SA 98/calibrated_r/RResult_1.50286457265.csv
Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/SA 98/calibrated_r/RResult_1.37231024482.csv


/var/folders/xh/69tqn8vn1vd5sw0_tn9ksy5w0000gn/T/ipykernel_46446/458396092.py:66: RuntimeWarning: invalid value encountered in log10
  'Instrumental_R': -2.5 * np.log10(row['aperture_sum_bkg_sub']),


## B Filter

In [2]:
import os
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
import astropy.units as u
from photutils.aperture import SkyCircularAperture, CircularAperture, SkyCircularAnnulus, aperture_photometry
from astropy.stats import sigma_clipped_stats

# Load APASS catalog CSV file
apass_data = pd.read_csv("/Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/SA 98/apassSA98.csv")

# Define your base path for FITS files
base_path = "/Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/SA 98/calibrated_b"

# Iterate over each file in the directory
for filename in os.listdir(base_path):
    if filename.endswith('.fts'):
        file_path = os.path.join(base_path, filename)
        results = []  # Initialize the results list inside the loop for separate CSV for each file
        try:
            hdulist = fits.open(file_path)
            data = hdulist[0].data
            wcs = WCS(hdulist[0].header)
            star_positions = SkyCoord(ra=apass_data['RA_degrees']*u.deg, dec=apass_data['Dec_degrees']*u.deg)

            # Retrieve airmass from the FITS header
            airmass = hdulist[0].header.get('AIRMASS', 'unknown')

            # Define apertures and convert to pixel coordinates
            apertures = SkyCircularAperture(star_positions, r=0.004 * u.deg)
            pixel_apertures = apertures.to_pixel(wcs)

            # Define annuli and convert to pixel coordinates for background estimation
            annuli = SkyCircularAnnulus(star_positions, r_in=0.005 * u.deg, r_out=0.006 * u.deg)
            pixel_annuli = annuli.to_pixel(wcs)

            # Perform background subtraction using sigma clipping
            bkg_median = []
            for annulus in pixel_annuli:
                mask = annulus.to_mask(method='center')
                annulus_data = mask.multiply(data)
                if np.any(mask.data > 0):  # Ensure data within mask
                    annulus_data_1d = annulus_data[mask.data > 0]
                    _, median_sigclip, _ = sigma_clipped_stats(annulus_data_1d)
                    bkg_median.append(median_sigclip)
                else:
                    bkg_median.append(0)  # No data to process

            bkg_median = np.array(bkg_median)
            phot_table = aperture_photometry(data, pixel_apertures, error=np.sqrt(data), wcs=wcs)
            phot_table['bkg_median'] = bkg_median
            phot_table['aperture_sum_bkg_sub'] = phot_table['aperture_sum'] - bkg_median * np.array([ap.area for ap in pixel_apertures])

            # Adding celestial coordinates
            phot_table['ra_icrs'] = star_positions.ra.degree
            phot_table['dec_icrs'] = star_positions.dec.degree

            # Match photometry results with catalog data
            for i, row in enumerate(phot_table):
                catalog_match = apass_data.iloc[np.argmin(star_positions.separation(star_positions[i]).degree)]
                results.append({
                    'RA': row['ra_icrs'],
                    'DEC': row['dec_icrs'],
                    'Instrumental_B': -2.5 * np.log10(row['aperture_sum_bkg_sub']),
                    'InstrumentalB_err': 1.0857 * row['aperture_sum_err'] / row['aperture_sum'],
                    'Known_B': catalog_match['B'],
                    'KnownB_err': catalog_match['B_err']
                })

            results_df = pd.DataFrame(results)
            csv_filename = f"BResult_{airmass}.csv"
            csv_file_path = os.path.join(base_path, csv_filename)
            results_df.to_csv(csv_file_path, index=False)
            print(f"Results saved to {csv_file_path}")

        except Exception as e:
            print(f"Failed to process {filename}: {e}")


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
/var/folders/xh/69tqn8vn1vd5sw0_tn9ksy5w0000gn/T/ipykernel_43427/3994092875.py:66: RuntimeWarning: invalid value encountered in log10
  'Instrumental_B': -2.5 * np.log10(row['aperture_sum_bkg_sub']),


Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/SA 98/calibrated_b/BResult_1.50498912204.csv


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
/var/folders/xh/69tqn8vn1vd5sw0_tn9ksy5w0000gn/T/ipykernel_43427/3994092875.py:66: RuntimeWarning: invalid value encountered in log10
  'Instrumental_B': -2.5 * np.log10(row['aperture_sum_bkg_sub']),


Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/SA 98/calibrated_b/BResult_1.77435329864.csv


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
/var/folders/xh/69tqn8vn1vd5sw0_tn9ksy5w0000gn/T/ipykernel_43427/3994092875.py:66: RuntimeWarning: invalid value encountered in log10
  'Instrumental_B': -2.5 * np.log10(row['aperture_sum_bkg_sub']),


...

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
/var/folders/xh/69tqn8vn1vd5sw0_tn9ksy5w0000gn/T/ipykernel_43427/3994092875.py:66: RuntimeWarning: invalid value encountered in log10
  'Instrumental_B': -2.5 * np.log10(row['aperture_sum_bkg_sub']),


Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/SA 98/calibrated_b/BResult_1.36141816757.csv


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
/var/folders/xh/69tqn8vn1vd5sw0_tn9ksy5w0000gn/T/ipykernel_43427/3994092875.py:66: RuntimeWarning: invalid value encountered in log10
  'Instrumental_B': -2.5 * np.log10(row['aperture_sum_bkg_sub']),
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/SA 98/calibrated_b/BResult_1.33751338933.csv
Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/SA 98/calibrated_b/BResult_1.46380866299.csv


/var/folders/xh/69tqn8vn1vd5sw0_tn9ksy5w0000gn/T/ipykernel_43427/3994092875.py:66: RuntimeWarning: invalid value encountered in log10
  'Instrumental_B': -2.5 * np.log10(row['aperture_sum_bkg_sub']),


## V Filter

In [39]:
import os
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
import astropy.units as u
from photutils.aperture import SkyCircularAperture, CircularAperture, SkyCircularAnnulus, aperture_photometry
from astropy.stats import sigma_clipped_stats

# Load APASS catalog CSV file
apass_data = pd.read_csv("/Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/SA 98/apassSA98.csv")

# Define your base path for FITS files
base_path = "/Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/SA 98/calibrated_v"

# Iterate over each file in the directory
for filename in os.listdir(base_path):
    if filename.endswith('.fts'):
        file_path = os.path.join(base_path, filename)
        results = []  # Initialize the results list inside the loop for separate CSV for each file
        try:
            hdulist = fits.open(file_path)
            data = hdulist[0].data
            wcs = WCS(hdulist[0].header)
            star_positions = SkyCoord(ra=apass_data['RA_degrees']*u.deg, dec=apass_data['Dec_degrees']*u.deg)

            # Retrieve airmass from the FITS header
            airmass = hdulist[0].header.get('AIRMASS', 'unknown')

            # Define apertures and convert to pixel coordinates
            apertures = SkyCircularAperture(star_positions, r=0.004 * u.deg)
            pixel_apertures = apertures.to_pixel(wcs)

            # Define annuli and convert to pixel coordinates for background estimation
            annuli = SkyCircularAnnulus(star_positions, r_in=0.005 * u.deg, r_out=0.006 * u.deg)
            pixel_annuli = annuli.to_pixel(wcs)

            # Perform background subtraction using sigma clipping
            bkg_median = []
            for annulus in pixel_annuli:
                mask = annulus.to_mask(method='center')
                annulus_data = mask.multiply(data)
                if np.any(mask.data > 0):  # Ensure data within mask
                    annulus_data_1d = annulus_data[mask.data > 0]
                    _, median_sigclip, _ = sigma_clipped_stats(annulus_data_1d)
                    bkg_median.append(median_sigclip)
                else:
                    bkg_median.append(0)  # No data to process

            bkg_median = np.array(bkg_median)
            phot_table = aperture_photometry(data, pixel_apertures, error=np.sqrt(data), wcs=wcs)
            phot_table['bkg_median'] = bkg_median
            phot_table['aperture_sum_bkg_sub'] = phot_table['aperture_sum'] - bkg_median * np.array([ap.area for ap in pixel_apertures])

            # Adding celestial coordinates
            phot_table['ra_icrs'] = star_positions.ra.degree
            phot_table['dec_icrs'] = star_positions.dec.degree

            # Match photometry results with catalog data
            for i, row in enumerate(phot_table):
                catalog_match = apass_data.iloc[np.argmin(star_positions.separation(star_positions[i]).degree)]
                results.append({
                    'RA': row['ra_icrs'],
                    'DEC': row['dec_icrs'],
                    'Instrumental_V': -2.5 * np.log10(row['aperture_sum_bkg_sub']),
                    'InstrumentalV_err': 1.0857 * row['aperture_sum_err'] / row['aperture_sum'],
                    'Known_V': catalog_match['V'],
                    'KnownV_err': catalog_match['V_err']
                })

            results_df = pd.DataFrame(results)
            csv_filename = f"VResult_{airmass}.csv"
            csv_file_path = os.path.join(base_path, csv_filename)
            results_df.to_csv(csv_file_path, index=False)
            print(f"Results saved to {csv_file_path}")

        except Exception as e:
            print(f"Failed to process {filename}: {e}")


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
/var/folders/xh/69tqn8vn1vd5sw0_tn9ksy5w0000gn/T/ipykernel_30578/4058917012.py:66: RuntimeWarning: invalid value encountered in log10
  'Instrumental_V': -2.5 * np.log10(row['aperture_sum_bkg_sub']),


Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/SA 98/calibrated_v/VResult_1.73625197253.csv


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
/var/folders/xh/69tqn8vn1vd5sw0_tn9ksy5w0000gn/T/ipykernel_30578/4058917012.py:66: RuntimeWarning: invalid value encountered in log10
  'Instrumental_V': -2.5 * np.log10(row['aperture_sum_bkg_sub']),
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/SA 98/calibrated_v/VResult_1.34628560154.csv


/var/folders/xh/69tqn8vn1vd5sw0_tn9ksy5w0000gn/T/ipykernel_30578/4058917012.py:66: RuntimeWarning: invalid value encountered in log10
  'Instrumental_V': -2.5 * np.log10(row['aperture_sum_bkg_sub']),
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


...

/var/folders/xh/69tqn8vn1vd5sw0_tn9ksy5w0000gn/T/ipykernel_30578/4058917012.py:66: RuntimeWarning: invalid value encountered in log10
  'Instrumental_V': -2.5 * np.log10(row['aperture_sum_bkg_sub']),
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/SA 98/calibrated_v/VResult_1.29636057409.csv


/var/folders/xh/69tqn8vn1vd5sw0_tn9ksy5w0000gn/T/ipykernel_30578/4058917012.py:66: RuntimeWarning: invalid value encountered in log10
  'Instrumental_V': -2.5 * np.log10(row['aperture_sum_bkg_sub']),


Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/SA 98/calibrated_v/VResult_1.48439362663.csv
Results saved to /Users/jonathan.a.chavez9/Desktop/greatBasinFilterCalibration/SA 98/calibrated_v/VResult_1.48271201828.csv


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
/var/folders/xh/69tqn8vn1vd5sw0_tn9ksy5w0000gn/T/ipykernel_30578/4058917012.py:66: RuntimeWarning: invalid value encountered in log10
  'Instrumental_V': -2.5 * np.log10(row['aperture_sum_bkg_sub']),
